# 집 근처 주유소 분석
- 정보는 크롤링을 통해 얻어오기(엑셀 다운 X)

In [1]:
import time
import pandas as pd 
import folium
import requests
from bs4 import BeautifulSoup
from urllib.parse import urlparse
from glob import glob 
from tqdm import tqdm_notebook
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

### 강서구, 양천구 데이터만 크롤링

In [28]:
driver = webdriver.Chrome('./chromedriver')
driver.get('http://www.opinet.co.kr/user/main/mainView.do')
time.sleep(1)
driver.find_element_by_css_selector('.ic_m1').click()
time.sleep(2)

region = driver.find_element_by_xpath('//*[@id="SIGUNGU_NM0"]')
gu_list = region.find_elements_by_tag_name('option')

gu_names = [gu.get_attribute('value') for gu in gu_list]
del gu_names[0]

# 강서구, 양천구에 대해서만 정보 크롤링
name_list = []; addr_list = []; trademark_list = []
g_price_list = []; d_price_list = []
for gu in tqdm_notebook(gu_names):
    region = driver.find_element_by_xpath('//*[@id="SIGUNGU_NM0"]')
    region.send_keys(gu)
    time.sleep(1)

    tbody = driver.find_element_by_id('body1')
    if gu == '강서구' or gu == '양천구':
        trs = tbody.find_elements_by_tag_name('tr')
        for tr in trs:
            tr.find_element_by_tag_name('a').click()
            time.sleep(1)
            info = driver.find_element_by_xpath('//*[@id="os_dtail_info"]/div[1]')
            name = info.find_element_by_xpath('//*[@id="os_nm"]').text
            addr = info.find_element_by_xpath('//*[@id="rd_addr"]').text
            trademark = info.find_element_by_xpath('//*[@id="poll_div_nm"]').text
            g_price = info.find_element_by_xpath('//*[@id="b027_p"]').text.replace(',', '')
            d_price = info.find_element_by_xpath('//*[@id="d047_p"]').text.replace(',', '')

            name_list.append(name); addr_list.append(addr)
            trademark_list.append(trademark)
            g_price_list.append(g_price); d_price_list.append(d_price)

In [29]:
len(name_list), len(addr_list), len(trademark_list), len(g_price_list), len(d_price_list)

(59, 59, 59, 59, 59)

In [35]:
print(d_price_list)

['1043', '1043', '1063', '1064', '', '1065', '1072', '1072', '1069', '', '1089', '1147', '1097', '1115', '1127', '1128', '1126', '1127', '1129', '1169', '1187', '1187', '1185', '1175', '1195', '1195', '1189', '1189', '1208', '1203', '1213', '1332', '1468', '1048', '1048', '1048', '1048', '1062', '1069', '1085', '1099', '1099', '1099', '1128', '1118', '1128', '1128', '1128', '1128', '1129', '1149', '1148', '1149', '1115', '1159', '1178', '1229', '1399', '1396']


In [37]:
stations = pd.DataFrame({
    '주유소명': name_list,
    '주소': addr_list,
    '상표': trademark_list,
    '휘발유': g_price_list,
    '경유': d_price_list
})
stations.head()

,주유소명,주소,상표,휘발유,경유
0,화곡역주유소,서울특별시 강서구 강서로 154 (화곡동),알뜰주유소,1243,1043
1,,,,1243,1043
2,,,,1263,1063
3,,,,1264,1064
4,,,,,


In [40]:
# '주유소명' 이 없는 행 제거
stations = stations[stations['주유소명'] != '']
stations.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 37 entries, 0 to 58
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   주유소명    37 non-null     object
 1   주소      37 non-null     object
 2   상표      37 non-null     object
 3   휘발유     37 non-null     object
 4   경유      37 non-null     object
dtypes: object(5)
memory usage: 1.7+ KB


In [51]:
# '주소' 를 기준으로 중복되는 행 제거 후 인덱스 초기화
stations = stations.drop_duplicates(['주소'])
stations.reset_index(drop=True, inplace=True)
stations.head()

,주유소명,주소,상표,휘발유,경유,위도,경도,color,icon
0,화곡역주유소,서울특별시 강서구 강서로 154 (화곡동),알뜰주유소,1243,1043,37.540768,126.841472,red,star
1,지에스칼텍스㈜ 경인고속주유소,서울 강서구 국회대로 225 (화곡동),GS칼텍스,1266,1072,37.529645,126.859835,blue,tint
2,목화주유소,서울 강서구 국회대로 251 (화곡동),SK에너지,1266,1072,37.530055,126.862834,blue,tint
3,스카이주유소,서울 강서구 개화동로 457 (방화동),S-OIL,1269,1069,37.571522,126.804526,blue,tint
4,KH에너지(주)직영 KH외발산주유소,서울 강서구 방화대로 30,SK에너지,1289,1089,37.543008,126.817064,blue,tint


In [52]:
key_fd = open('kakaomapkey.txt', mode='r')
kmap_key = key_fd.read(100)
key_fd.close()

In [53]:
lat_list = []; lng_list = []
for i in tqdm_notebook(stations.index):
    url = 'https://dapi.kakao.com/v2/local/search/address.json?query=' + stations['주소'][i]
    result = requests.get(urlparse(url).geturl(), headers={"Authorization":"KakaoAK " + kmap_key}).json()
    if result['documents'] == []:
        lat_list.append(np.nan); lng_list.append(np.nan)
    else:
        lat = float(result['documents'][0]['y'])
        lng = float(result['documents'][0]['x'])
        lat_list.append(lat); lng_list.append(lng)

In [54]:
len(stations['주유소명']), len(stations['주소']), len(stations['상표']), len(stations['휘발유']), len(stations['경유'])

(37, 37, 37, 37, 37)

In [55]:
stations['위도'] = lat_list
stations['경도'] = lng_list
stations.head()

,주유소명,주소,상표,휘발유,경유,위도,경도,color,icon
0,화곡역주유소,서울특별시 강서구 강서로 154 (화곡동),알뜰주유소,1243,1043,37.540768,126.841472,red,star
1,지에스칼텍스㈜ 경인고속주유소,서울 강서구 국회대로 225 (화곡동),GS칼텍스,1266,1072,37.529645,126.859835,blue,tint
2,목화주유소,서울 강서구 국회대로 251 (화곡동),SK에너지,1266,1072,37.530055,126.862834,blue,tint
3,스카이주유소,서울 강서구 개화동로 457 (방화동),S-OIL,1269,1069,37.571522,126.804526,blue,tint
4,KH에너지(주)직영 KH외발산주유소,서울 강서구 방화대로 30,SK에너지,1289,1089,37.543008,126.817064,blue,tint


In [56]:
stations['color'] = 'blue'; stations['icon'] = 'tint'

In [57]:
# 휘발유 가격을 기준으로 가장 저렴한 곳 찾은 후 color와 icon 다르게 표시
top1 = stations.sort_values(by='휘발유', ascending=True).head(1)
top1['color'] = 'red'; top1['icon'] = 'star'
top1

,주유소명,주소,상표,휘발유,경유,위도,경도,color,icon
0,화곡역주유소,서울특별시 강서구 강서로 154 (화곡동),알뜰주유소,1243,1043,37.540768,126.841472,red,star


In [58]:
# 원래 데이터프레임과 병합 후 중복제거
# 단, top1을 앞에 놓음으로써 제거되지 않게하고 원래 있던 행 제거
stations = pd.concat([top1, stations])
stations = stations.drop_duplicates(['주소'])
stations.reset_index(drop=True, inplace=True)
stations.head()

,주유소명,주소,상표,휘발유,경유,위도,경도,color,icon
0,화곡역주유소,서울특별시 강서구 강서로 154 (화곡동),알뜰주유소,1243,1043,37.540768,126.841472,red,star
1,지에스칼텍스㈜ 경인고속주유소,서울 강서구 국회대로 225 (화곡동),GS칼텍스,1266,1072,37.529645,126.859835,blue,tint
2,목화주유소,서울 강서구 국회대로 251 (화곡동),SK에너지,1266,1072,37.530055,126.862834,blue,tint
3,스카이주유소,서울 강서구 개화동로 457 (방화동),S-OIL,1269,1069,37.571522,126.804526,blue,tint
4,KH에너지(주)직영 KH외발산주유소,서울 강서구 방화대로 30,SK에너지,1289,1089,37.543008,126.817064,blue,tint


### 지도 위에 주유소 10곳 표시
- 가장 저렴한 주유소는 따로 표시

In [65]:
map = folium.Map(location=[stations['위도'].mean(), stations['경도'].mean()], zoom_start=13)

for i in stations.index:
    folium.Marker(
        location=[stations['위도'][i], stations['경도'][i]],
        popup=stations['주소'][i],
        tooltip=f"휘발유 가격: {stations['휘발유'][i]}원 / 경유 가격: {stations['경유'][i]}원",
        icon=folium.Icon(color=stations.color[i], icon=stations.icon[i])
    ).add_to(map)
map